In [ ]:
import yfinance as yf
import pandas as pd
from newsapi import NewsApiClient
from datetime import datetime, timedelta

In [ ]:
api_key = ''
newsapi = NewsApiClient(api_key=api_key)

In [91]:
def get_stock_and_news_data(tickers, start_date, end_date):
    all_stock_data = []
    all_news_data = []

    for ticker in tickers:
        # Stock price data
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['PriceChange'] = stock_data['Close'].pct_change()
        stock_data = stock_data.dropna()
        stock_data['Movement'] = stock_data['PriceChange'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
        stock_data.index = pd.to_datetime(stock_data.index).date
        stock_data['stock'] = ticker
        all_stock_data.append(stock_data)
        
        # News headlines
        headlines = newsapi.get_everything(q=f'{ticker} stock', from_param=start_date, to=end_date, language='en', sort_by='relevancy', page_size=100)
        
        # Convert news articles to DataFrame
        headlines_df = pd.DataFrame(headlines['articles'])
        headlines_df = headlines_df[headlines_df['title'] != '[Removed]']  # Discard news with title [Removed]
        headlines_df['publishedAt'] = pd.to_datetime(headlines_df['publishedAt']).dt.date
        headlines_df['content'] = headlines_df['content'].fillna('')
        headlines_df['stock'] = ticker
        all_news_data.append(headlines_df)
    
    # Concatenate all dataframes
    stock_df = pd.concat(all_stock_data)
    news_df = pd.concat(all_news_data)
    
    return stock_df, news_df

[*********************100%%**********************]  1 of 1 completed


/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6718/2971109579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Movement'] = stock_data['PriceChange'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6718/2971109579.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['stock'] = ticker
[*********************100%%**********************]  1 of 1 completed
/var/folders/wm/c5dyqf8x7gg

In [ ]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
start_date = "2024-04-25"
end_date = "2024-05-23"

In [ ]:
stock_df, news_df = get_stock_and_news_data(tickers, start_date, end_date)

In [96]:
stock_df

,Open,High,Low,Close,Adj Close,Volume,PriceChange,Movement,stock
2024-04-26,169.880005,171.339996,169.179993,169.300003,169.070679,44838400,-0.003473,Negative,AAPL
2024-04-29,173.369995,176.029999,173.100006,173.500000,173.264984,68169400,0.024808,Positive,AAPL
2024-04-30,173.330002,174.990005,170.000000,170.330002,170.099289,65934800,-0.018271,Negative,AAPL
2024-05-01,169.580002,172.710007,169.110001,169.300003,169.070679,50383100,-0.006047,Negative,AAPL
2024-05-02,172.509995,173.419998,170.889999,173.029999,172.795624,94214900,0.022032,Positive,AAPL
...,...,...,...,...,...,...,...,...,...
2024-05-16,174.100006,175.789993,171.429993,174.839996,174.839996,59812200,0.004885,Positive,TSLA
2024-05-17,173.550003,179.630005,172.750000,177.460007,177.460007,77445800,0.014985,Positive,TSLA
2024-05-20,177.559998,177.750000,173.520004,174.949997,174.949997,61727400,-0.014144,Negative,TSLA
2024-05-21,175.509995,186.880005,174.710007,186.600006,186.600006,115266500,0.066591,Positive,TSLA


In [95]:
news_df

,source,author,title,description,url,urlToImage,publishedAt,content,stock
0,"{'id': None, 'name': '9to5Mac'}",Ben Lovejoy,Tomorrow’s AAPL Q2 earnings likely to be $5B down,AAPL Q2 earnings are set to be announced tomor...,https://9to5mac.com/2024/05/01/aapl-q2-earning...,https://i0.wp.com/9to5mac.com/wp-content/uploa...,2024-05-01,AAPL Q2 earnings are set to be announced tomor...,AAPL
1,"{'id': None, 'name': 'AppleInsider'}",news@appleinsider.com (Mike Wuerthele),"One hedge fund completely bailed out of AAPL, ...",Prior to Apple stock's value recovery after a ...,https://appleinsider.com/articles/24/05/16/one...,https://photos5.appleinsider.com/gallery/58327...,2024-05-16,Apple CEO Tim Cook\r\nPrior to Apple stock's v...,AAPL
2,"{'id': None, 'name': 'Investor's Business Daily'}",Investor's Business Daily,Google I/O Event Expectations Run High Amid Op...,Expectations for Google stock run high ahead o...,https://www.investors.com/news/technology/goog...,https://www.investors.com/wp-content/uploads/2...,2024-05-14,Expectations run high ahead of Alphabet's (GOO...,AAPL
3,"{'id': None, 'name': '9to5Mac'}",Chance Miller,Warren Buffett’s Berkshire Hathaway sells 13% ...,Warren Buffett’s Berkshire Hathaway offloaded ...,https://9to5mac.com/2024/05/04/warren-buffett-...,https://i0.wp.com/9to5mac.com/wp-content/uploa...,2024-05-04,Warren Buffett’s Berkshire Hathaway offloaded ...,AAPL
4,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Will Rising Margins And Stock Buybacks Drive A...,While revenue fell by 4% year-over-year to $90...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-05-06,"MADRID, SPAIN - 2024/04/21: Shoppers walk into...",AAPL
...,...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",What’s Next For JetBlue Stock After A Sharp 19...,JetBlue (NASDAQ: JBLU) recently reported its Q...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-04-25,"JetBlue Airbus A321neo aircraft flying, landin...",TSLA
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Should You Pick CVS Health Stock At $55 After ...,CVS Health (NYSE: CVS) recently reported its Q...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-05-03,"LAS VEGAS, NEVADA - FEBRUARY 26: Signs for an ...",TSLA
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Can A New Model Y Challenger Get Nio Stock Bac...,The new vehicle will have a starting price of ...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-05-20,"SHANGHAI, CHINA - MAY 15: ONVO L60 electric SU...",TSLA
98,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Will Medtronic Stock See Higher Levels Post Q4...,The company should continue to benefit from an...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-05-21,"This photo taken on Nov. 8, 2023 shows a booth...",TSLA
